In [1]:
# Packages
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time

In [4]:
# ! python -m spacy download de

     |████████████████████████████████| 11.1 MB 133 kB/s eta 0:00:01
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.1.0-py3-none-any.whl size=11073066 sha256=9bdae4b917cab9f06d7c743b175ac0f950035d46056c7630f7c22246b1f8c12d
  Stored in directory: /tmp/pip-ephem-wheel-cache-yapx0c5f/wheels/19/c6/ac/c9e47e5851255b175d864b74fef866e331c8375a5517a912cb
Successfully built de-core-news-sm
  Attempting uninstall: de-core-news-sm
    Found existing installation: de-core-news-sm 2.2.5
    Uninstalling de-core-news-sm-2.2.5:
      Successfully uninstalled de-core-news-sm-2.2.5
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/home/vishal/Desktop/pytorch_gpu/pytorch_gpu/lib/python3.7/site-packages/de_core_news_sm
-->
/home/vishal/Desktop/pytorch_gpu/pytorch_gpu/lib/python3.7/site-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [6]:
# ! python -m spacy download en

     |████████████████████████████████| 11.1 MB 125 kB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.1.0-py3-none-any.whl size=11074433 sha256=e03ca909e6758da794722a633f59ba13f600af9add4c22577d6f30cad73916e6
  Stored in directory: /tmp/pip-ephem-wheel-cache-ve4z2h3u/wheels/59/4f/8c/0dbaab09a776d1fa3740e9465078bfd903cc22f3985382b496
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/vishal/Desktop/pytorch_gpu/pytorch_gpu/lib/python3.7/site-packages/en_core_web_sm
-->
/home/vishal/Desktop/pytorch_gpu/pytorch_gpu/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [7]:
# Create the tokenizers
spacy_de=spacy.load('de')
spacy_en=spacy.load('en')

In [8]:
def tokenize_de(text):
    """ Tokenizes german text from a string into a list of strings """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """ Tokenizes english text from a string into a list of strings """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [9]:
# The model expects data to be fed in with batch dimension first, so we use batch_first=True

In [10]:
SRC=Field(tokenize=tokenize_de, init_token='<sos>', eos_token='<eos>', lower=True, batch_first=True)

TRG=Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<sos>', lower=True, batch_first=True)

In [11]:
# We load the Multi30k dataset and build the vocabulary

In [12]:
train_data, valid_data, test_data=Multi30k.splits(exts=('.de', '.en'), fields=(SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:04<00:00, 255kB/s] 


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 113kB/s] 


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 103kB/s] 


In [13]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

In [14]:
# Finally we define the device and the data iterator

In [15]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
BATCH_SIZE=64

# train_iterator, valid_iterator, test_iterator=BucketIterator.splits((train_data, valid_data, test_data), batch_sizes=BATCH_SIZE, device=device)
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
     batch_size = BATCH_SIZE,
     device = device)

# Building the model

## Encoder

In [35]:
torch.arange(0, 100).unsqueeze(0).repeat(64, 1).shape

torch.Size([64, 100])

In [36]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hid_dim, n_layers, n_heads, pf_dim, dropout, device, max_length=100):
        super.__init()
        
        self.device=device
        self.tok_embedding=nn.Embedding(input_dim, output_dim)
        self.pos_embedding=nn.Embedding(max_length, hid_dim)
        
        self.layers=nn.ModuleList([EncoderLayer(hid_dim, n_heads, pf_dim, dropout, device) for _ in range(n_layers)])
        
        self.dropout=nn.Dropout(dropout)
        
        self.scale=torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        # src= [batch_size, src_len]
        # src_mask=[batch_size, src_len]
        
        batch_size=src.shape[0]
        src_len=src.shape[1]
        
        pos=torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        # pos=[batch_size, src_len]
        
        src=self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        # src=[batch_size, src_len, hid_dim]
        
        for layer in self.layers:
            src=layer(src, src_mask)
        
        # src=[batch_size, src_len, hid_dim]
        
        return src